## CESM2 - LARGE ENSEMBLE (LENS2)
- In this Notebook we want to control the temperature flow: speed and temperature. For velocity, we will compute the advective terms and the eddies. 

### Imports

In [ ]:
# modules I am using in this example
import xarray as xr
import xgcm
from xgcm import Grid
import pop_tools
from dask.distributed import Client, wait
from ncar_jobqueue import NCARCluster
import dask
import intake
import intake_esm

### Dask

In [ ]:
mem_per_worker = 40 # memory per worker in GB 
num_workers = 40 # number of workers
cluster = NCARCluster(cores=1, processes=1, memory=f'{mem_per_worker} GB',resource_spec=f'select=1:ncpus=1:mem={mem_per_worker}GB', walltime='2:00:00')
cluster.scale(num_workers)
client = Client(cluster)
print(client)
client

### Load data

In [ ]:
%%time
path = '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/UET/b.e21.BSSP370smbb.f09_g17.LE2-1301.019.pop.h.UET.205501-206412.nc'
ds_UET = xr.open_mfdataset(path,parallel=True)
path = '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/VNT/b.e21.BSSP370smbb.f09_g17.LE2-1301.019.pop.h.VNT.205501-206412.nc'
ds_VNT = xr.open_mfdataset(path,parallel=True)
path = '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/VVEL/b.e21.BSSP370smbb.f09_g17.LE2-1301.019.pop.h.VVEL.205501-206412.nc'
ds_VVEL = xr.open_mfdataset(path,parallel=True)
path = '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/UVEL/b.e21.BSSP370smbb.f09_g17.LE2-1301.019.pop.h.UVEL.205501-206412.nc'
ds_UVEL = xr.open_mfdataset(path,parallel=True)
path = '/glade/campaign/cgd/cesm/CESM2-LE/timeseries/ocn/proc/tseries/month_1/TEMP/b.e21.BSSP370smbb.f09_g17.LE2-1301.019.pop.h.TEMP.205501-206412.nc'
ds_TEMP = xr.open_mfdataset(path,parallel=True)
del path
ds = xr.merge([ds_UET,ds_VNT,ds_VVEL,ds_UVEL,ds_TEMP],compat='override')
del ds_UET,ds_VNT,ds_VVEL,ds_UVEL,ds_TEMP

### Concatenation of variables

In [ ]:
%%time
# Cutting out and centering the variables in the South Atlantic
dask.config.set({"array.slicing.split_large_chunks": True})
ilon1, flon1, ilon2, flon2 = 307, 320, 0, 54 # longitude (initial (i), final (f)) 
ilan = 0 # northernmost latitude
ilas = -34 # southernmost latitude

ds=xr.combine_nested([[
    ds.where((ds.TLAT >= ilas) & (ds.TLAT <= ilan), drop=True).isel(
        nlon = slice(ilon1,flon1)),
    ds.where((ds.TLAT >= ilas) & (ds[f'{var}'].TLAT <= ilan), drop=True).isel(
        nlon = slice(ilon2,flon2))]],
    concat_dim=['nlat','nlon'])   
ds.coords['nlon'] = (ds.coords['nlon'] + 180) % 360 - 180 
ds = ds.sortby(ds.nlon)

In [ ]:
ds.dz

### Help functions

In [ ]:
def to_index(ds):
    ds = ds.copy()
    for dim in ds.dims:
        if dim in ["nlon_t", "nlat_t", "nlon_u", "nlat_u"]:
            ds = ds.drop(dim).rename({dim: dim[:-2]})
    return ds
def to_coord(ds, lon, lat):
    return ds.rename({"nlon": "nlon_" + lon,
                      "nlat": "nlat_" + lat})

### Get the POP_grid

In [ ]:
# get the cell volume and the grid from pop_tools. this should be easier for LENS low res, as your DZT is just your dz because you are not dealing with partial bottom cells (right Gustavo?) 

ds['cell_volume'] = ds.dz * ds.DXT * ds.DYT # Volume (check out later the unit)
grid, ds_ren = pop_tools.to_xgcm_grid_dataset(ds) # We gotta confirm if we may use just one variable to compute the grid and ds_ren

### Advection

In [ ]:
ds_ren

In [ ]:
%%time
# get the total advection as saved by the model 
# total advection saved by model --> the difference betweem this and the mean is the eddy component
# horizontal components
print('Getting total advection term.')
#st = time.time()
uadv = -( grid.diff(to_coord((ds_ren.cell_volume * ds.UET), 'u', 't'), 
                   axis="X", boundary="extend")
        / ds_ren.cell_volume )
vadv = -( grid.diff(to_coord((ds_ren.cell_volume * ds.VNT), 't', 'u'),
                   axis="Y", boundary="extend")
        / ds_ren.cell_volume )

In [ ]:
uadv

In [ ]:
# total horizontal
h_adv = uadv + vadv # I think you only want vnt? just copying everything here for completeness 

In [ ]:
# get the advection from the mean flow 
# horizontal advection from mean flow -- you need this because the difference between this and the term above is the eddy term 
print('Getting horizontal advection from mean flow.')
#st = time.time()
# u term
U_interp = grid.interp((ds_ren.UVEL * ds_ren.dz * ds_ren.DYU),
                      axis="Y",
                      boundary="extend")
uT = U_interp * grid.interp(ds_ren.TEMP,
                            axis="X",
                            boundary="extend")
H_ADV_mean = (-(grid.diff(uT, axis="X", boundary="extend")
               / ds_ren.cell_volume)).load()
# v term
V_interp = grid.interp((ds_ren.VVEL * ds_ren.dz * ds_ren.DXU),
                       axis="X", boundary="extend")
vT = V_interp * grid.interp(ds_ren.TEMP,
                            axis="Y", boundary="extend")
# total term, again you don't need this, you just need the V term I think, and you do want to do the operation grid.diff(...) / cell_volume
H_ADV_mean = H_ADV_mean - (grid.diff(vT, axis="Y", boundary="extend")
                           / ds_ren.cell_volume)

In [ ]:
#then calculate difference for eddy part  
T_h_ADV_eddy = h_adv - H_ADV_mean